In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import os
import glob
from Action_Segmentation import *
from Feature_Engineer import *
import matplotlib.pyplot as plt
from datetime import timedelta
from datetime import time
from sklearn.pipeline import Pipeline
os.chdir("C:\\Users\\Andrius\\Downloads\\Mouse-Dynamics-Challenge-master\\Mouse-Dynamics-Challenge-master")
train=pd.read_csv('train_save_pre_seg.csv')

In [3]:
train.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'record timestamp', 'client timestamp',
       'button', 'state', 'x', 'y', 'session', 'user'],
      dtype='object')

In [20]:
import numpy as np
import pandas as pd

def action_row(row):
    if row['state']=='Released' and row['state_lag']=='Drag':
        return 'DD'
    elif row['state']=='Released' and row['state_lag']=='Pressed':
        return 'PC'
    
def delta_maker(data):
    temp=pd.DataFrame()
    
    temp=data[(data['Action']=='DD') | (data['Action']=='PC') | (data['Action']=='MM')]
    
    data.loc[:,'action_begin']=temp['client timestamp'].shift()
    data.loc[data.index[0],'action_begin']=data['client timestamp'].values[0]
    data.loc[temp['client timestamp'].index[0],'action_begin']=data['client timestamp'].iloc[0]
    
    data.loc[:,'delta_action']=temp['client timestamp']-data['action_begin']
    
    
    temp['index']=temp.index
    
    temp['begin_act_idx']=temp['index'].shift()
    
    temp.loc[temp.index[0],'begin_act_idx']=data.index[0]
    
    temp.set_index(temp['index'],inplace=True)
    
    #temp.columns.name=None
    data.loc[:,'begin_act_idx']=data.merge(temp,how='left',left_index=True, right_index=True)['begin_act_idx']
   
def post_seg_delta_maker(data):
    
    data.loc[:,'action_begin']=data[(data['Action_border']=='DD') | (data['Action_border']=='PC') | (data['Action_border']=='MM')]['client timestamp'].shift()
    
    data.loc[data[(data['Action_border']=='DD') | (data['Action_border']=='PC') | (data['Action_border']=='MM')]['client timestamp'].index[0],'action_begin']=data['client timestamp'].iloc[0]
    #data[(data['Action']=='DD') | (data['Action']=='PC')].reset_index(drop=True).loc[0,'action_begin']=data['client timestamp'].loc[0]
    data.loc[:,'delta_action']=data[(data['Action_border']=='DD') | (data['Action_border']=='PC') | (data['Action_border']=='MM')]['client timestamp']-data['action_begin']
    
def closest_timestamp(row,data,min_events):
    
    temp=(data['client timestamp']-row['action_corrector']).abs().sort_values().index[0]
    
    #Considerar minimo de eventos
    if row.name-temp >= min_events :
        data.loc[temp,'Action']='MM'
        data.loc[row.name,'act_correc_index']=temp
    
        
def action_mm_finder(data,thresh,min_events):    
    # 1- Calcular time -resto(delta/10)
    data['action_corrector']=data['client timestamp']-data['delta_action']%thresh
    
    #-2 Função para para procurar índice da linha mais proxima do valor
    data[data['delta_action']>=thresh].apply(closest_timestamp,args=(data,min_events),axis=1)
    
        
def correct_small_actions(row,min_events):
    
    if row.Action_border!=None and row.Action_border_lag!=None :

        return row.Action_border_lag
    else:
        if row.name-row.begin_act_idx < min_events:
            return None
        else:
            return row.Action_border
    

def correct_small_actions_2(row):
    
    if row.Action_border!=None and row.Action_border_antilag!=None and pd.notna(row.Action_border_antilag): 
        return None
    else:
        return row.Action_border
    

    
def segmentation(data2,thresh,min_events):
    
    data=data2.copy(deep=True)
    data['state_lag']=data['state'].shift()
    data.loc[:,'Action']=data.apply(action_row,axis=1)
    delta_maker(data)
    action_mm_finder(data,thresh,min_events)
    if data.Action.iloc[[-1]].item() != 'PC' and data.Action.iloc[[-1]].item() != 'DD':
        data.loc[data.index[-1],'Action']='MM'

    
    data['Action_border']=data['Action']
    data['Action_border_lag']=data['Action_border'].shift()
    data['Action_border']=data.apply(correct_small_actions,args=(min_events,),axis=1)
    
    data['Action_border_antilag']=data['Action_border'].shift(-1)
    data['Action_border']=data.apply(correct_small_actions_2,axis=1)
    data.loc[data.index[-1],'Action_border']=data.Action.iloc[[-1]].values[0]
    data['Action']=data.Action.fillna(method='bfill')
    data.drop(['Action_border_lag','Action_border_antilag'],axis=1,inplace=True)
    
    return data


#Função para tratar train_seg:
def segmentation_prep(data):
    data=data.drop(['state_lag','action_corrector'],axis=1)

    post_seg_delta_maker(data)
    return data

In [21]:
train_seg=pd.DataFrame()
thresh=10
min_events=4


train_seg=train_seg.append(segmentation(train[train.session=='session_1658051584'],thresh,min_events))
train_seg=segmentation_prep(train_seg)
train_save=train_seg
#train_seg=segmentation_prep(train_seg)

<ipython-input-20-10518e831677>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['index']=temp.index
<ipython-input-20-10518e831677>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['begin_act_idx']=temp['index'].shift()
C:\Users\Andrius\anaconda\envs\data analysis\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [ ]:
train_seg

In [22]:
train_seg.Action_border

488329    None
488330    None
488331    None
488332    None
488333    None
          ... 
525788    None
525789    None
525790      PC
525791    None
525792      PC
Name: Action_border, Length: 37464, dtype: object

In [7]:
train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'x']

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Float64Index([nan, nan], dtype='float64'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [23]:
train_seg=train_save
temp=pd.DataFrame()
#train_seg=train_seg[train_seg.session=='session_5265929106']
temp=pd.DataFrame((train_seg[(train_seg.Action_border=='PC')| (train_seg.Action_border=='MM') | (train_seg.Action_border=='DD')].Action_border.index))
temp['action_count']=temp.index
temp['index']=temp.loc[:,0]
temp.drop(0,axis=1,inplace= True)
temp.set_index(temp['index'],inplace=True)
temp.columns.name=None
train_seg=train_seg.merge(temp,how='left',left_index=True, right_index=True)
train_seg['action_count'].fillna(method='bfill',inplace=True)
#train_seg.drop('index',axis=1,inplace=True)
#2nd dataframe with action features
df_action=train_seg[train_seg.Action_border.notna()].copy(deep=True)
df_action['index']=train_seg[train_seg.Action_border.notna()].index
df_action['begin_act_idx']=df_action['index'].shift()
df_action.loc[df_action.index[0],'begin_act_idx']=train_seg.index[0]
train_seg.loc[:,'begin_act_idx']=df_action['begin_act_idx']
    ###########################################################
    #############################################################
train_seg.loc[:,'begin_act_idx'].fillna(method='bfill',inplace=True)

In [24]:
train_seg['begin_act_idx']

488329    488329.0
488330    488329.0
488331    488329.0
488332    488329.0
488333    488329.0
            ...   
525788    525764.0
525789    525764.0
525790    525764.0
525791    525790.0
525792    525790.0
Name: begin_act_idx, Length: 37464, dtype: float64

In [22]:
df_action['begin_act_idx']

488478    488329.0
488506    488478.0
488535    488506.0
488545    488535.0
488553    488545.0
            ...   
525737    525727.0
525743    525737.0
525751    525743.0
525764    525751.0
525790    525764.0
Name: begin_act_idx, Length: 2027, dtype: float64

In [9]:
train_seg.Action_border

488329    None
488330    None
488331    None
488332    None
488333    None
          ... 
525788    None
525789    None
525790      PC
525791    None
525792    None
Name: Action_border, Length: 37464, dtype: object

In [187]:
def Euclidean_Dist(df1, df2):
    return np.linalg.norm(df1.values - df2.values,
                   axis=1)
#Função distancia segmento/ponto
def lineseg_dists(p, a, b):
    """Cartesian distance from point to line segment

    Edited to support arguments as series, from:
    https://stackoverflow.com/a/54442561/11208892

    Args:
        - p: np.array of single point, shape (2,) or 2D array, shape (x, 2)
        - a: np.array of shape (x, 2)
        - b: np.array of shape (x, 2)
    """
    # normalized tangent vectors
    d_ba = b - a
    d = np.divide(d_ba, (np.hypot(d_ba[:, 0], d_ba[:, 1])
                           .reshape(-1, 1)))
    #np.nan_to_num(d,copy=False,nan=[0,0])
    #print(np.isnan(d).sum())

    # signed parallel distance components
    # rowwise dot products of 2D vectors
    s = np.multiply(a - p, d).sum(axis=1)
    t = np.multiply(p - b, d).sum(axis=1)

    # clamped parallel distance
    h = np.maximum.reduce([s, t, np.zeros(len(s))])

    # perpendicular distance component
    # rowwise cross products of 2D vectors  
    d_pa = p - a
    #print(c)
    c = d_pa[:, 0] * d[:, 1] - d_pa[:, 1] * d[:, 0]

    return np.hypot(h, c)
def feat_eng(train_seg):
    temp=pd.DataFrame()
    #train_seg=train_seg[train_seg.session=='session_5265929106']
    temp=pd.DataFrame((train_seg[(train_seg.Action_border=='PC')| (train_seg.Action_border=='MM') | (train_seg.Action_border=='DD')].Action_border.index))
    temp['action_count']=temp.index
    temp['index']=temp.loc[:,0]
    temp.drop(0,axis=1,inplace= True)
    temp.set_index(temp['index'],inplace=True)
    temp.columns.name=None
    train_seg=train_seg.merge(temp,how='left',left_index=True, right_index=True)
    train_seg['action_count'].fillna(method='bfill',inplace=True)
    #train_seg.drop('index',axis=1,inplace=True)

    #2nd dataframe with action features

    df_action=train_seg[train_seg.Action_border.notna()].copy(deep=True)
    df_action['index']=train_seg[train_seg.Action_border.notna()].index
    df_action['begin_act_idx']=df_action['index'].shift()
    df_action['action_count']=train_seg['action_count']
    df_action.loc[df_action.index[0],'begin_act_idx']=train_seg.index[0]
    train_seg.loc[:,'begin_act_idx']=df_action['begin_act_idx']
    ###########################################################
    #############################################################
    train_seg.loc[:,'begin_act_idx'].fillna(method='bfill',inplace=True)
    train_seg.loc[:,'end_act_idx']=df_action['index']
    train_seg.loc[:,'end_act_idx'].fillna(method='bfill',inplace=True)
    df_action.loc[df_action.index.to_list()[0],'begin_act_idx']=0


    #delta x,y, t
    train_seg['delta_x']=train_seg['x']-train_seg['x'].shift()
    train_seg['delta_y']=train_seg['y']-train_seg['y'].shift()
    train_seg.loc[train_seg.index[0],'delta_x']=0
    train_seg.loc[train_seg.index[0],'delta_y']=0
    #print(train_seg.loc[:,'begin_act_idx'].values)
    #train_seg['delta_x_begin_act']=np.sqrt((train_seg['x'].values-train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'x'].values)**2+(train_seg['y'].values-train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'y'].values)**2)
    #train_seg['delta_x_end_act']=np.sqrt((train_seg['x'].values-train_seg.loc[train_seg.loc[:,'end_act_idx'].values,'x'].values)**2+(train_seg['y'].values-train_seg.loc[train_seg.loc[:,'end_act_idx'].values,'y'].values)**2)
    #print(np.array(list(zip(train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'x'],train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'y']))))
    train_seg['deviation']=lineseg_dists(p=np.array(train_seg[['x','y']]),
                                         a=np.array(list(zip(train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'x'],train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'y']))),
                                         b=np.array(list(zip(train_seg.loc[train_seg.loc[:,'end_act_idx'].values,'x'],train_seg.loc[train_seg.loc[:,'end_act_idx'].values,'y']))))  
    
    train_seg['dist_from_act_init']=Euclidean_Dist(train_seg.loc[train_seg['begin_act_idx'],['x','y']],
    train_seg[['x','y']])
    train_seg.deviation.fillna(train_seg['dist_from_act_init'],inplace=True)
    
    
    #Tempo
    train_seg['delta_t']=(train_seg['client timestamp']-train_seg['client timestamp'].shift())
    train_seg['delta_t'].replace(0, train_seg[train_seg['delta_t']!=0].delta_t.min()/2,inplace=True)
    train_seg.loc[train_seg.index[0],'delta_t']=0
    
    #Velo
    train_seg['vel_x']=train_seg['delta_x']/train_seg['delta_t']
    train_seg.loc[train_seg.index[0],'vel_x']=0    
    train_seg['vel_x_delta']=train_seg['vel_x']-train_seg['vel_x'].shift()
    train_seg.loc[train_seg.index[0],'vel_x_delta']=0    
    #velocidades 
    train_seg['vel_y']=train_seg['delta_y']/train_seg['delta_t']
    train_seg.loc[train_seg.index[0],'vel_y']=0    
    train_seg.loc[:,'vel_y'].replace([np.inf, -np.inf], np.nan, inplace=True)
    train_seg['vel_y_delta']=train_seg['vel_y']-train_seg['vel_y'].shift()
    train_seg.loc[train_seg.index[0],'vel_y_delta']=0    
    train_seg['vel_total']=np.sqrt(np.power(train_seg['vel_x'],2)+np.power(train_seg['vel_y'],2))
    train_seg.loc[train_seg.index[0],'vel_total']=0    
    train_seg['vel_tot_delta']=train_seg['vel_total']-train_seg['vel_total'].shift()
    train_seg.loc[train_seg.index[0],'vel_tot_delta']=0
    #theta coordenada angular
    train_seg['theta']=np.arctan(train_seg['delta_y']/train_seg['delta_x'])
    train_seg['theta'].replace(np.nan,0, inplace=True)
    train_seg.loc[train_seg.index[0],'theta']=0
    train_seg['theta_delta']=train_seg['theta']-train_seg['theta'].shift()
    train_seg.loc[train_seg.index[0],'theta_delta']=0 
    #extras
    train_seg['acc']=train_seg['vel_tot_delta']/train_seg['delta_t']
    train_seg.loc[train_seg.index[0],'acc']=0 
    train_seg['acc_delta']=train_seg['acc']-train_seg['acc'].shift()
    train_seg.loc[train_seg.index[0],'acc_delta']=0  
    train_seg['jerk']=train_seg['acc_delta']/train_seg['delta_t']
    train_seg.loc[train_seg.index[0],'jerk']=0
    #velocidade angular
    train_seg['vel_angular']=train_seg['theta_delta']/train_seg['delta_t']
    train_seg.loc[train_seg.index[0],'vel_angular']=0

    #Curvature
    train_seg['event_length']=np.sqrt(np.power(train_seg['delta_x'],2)+np.power(train_seg['delta_y'],2))
    train_seg['cum_length']=train_seg['event_length'].cumsum()  

    train_seg['cum_length_delta']=train_seg['cum_length']-train_seg['cum_length'].shift()
    train_seg.loc[train_seg.index[0],'cum_length_delta']=0   

    train_seg['curvat']=train_seg['theta_delta']/train_seg['delta_t']
    train_seg.loc[train_seg.index[0],'curvat']=0
    #Action Dataframe
    #df_action=train_seg[train_seg.Action_border.notna()].copy(deep=True)
    #df_action['begin_act_idx']=[0]+df_action.index.to_list()
    df_action['v_x_mean']=df_action.merge(train_seg.groupby(by='action_count')['vel_x'].mean(),left_on=['action_count'],right_index=True,how='left')['vel_x']
    df_action['v_x_std']=df_action.merge(train_seg.groupby(by='action_count')['vel_x'].std(),left_on=['action_count'],right_index=True,how='left')['vel_x']
    df_action['v_x_max']=df_action.merge(train_seg.groupby(by='action_count')['vel_x'].max(),left_on=['action_count'],right_index=True,how='left')['vel_x']
    df_action['v_x_min']=df_action.merge(train_seg.groupby(by='action_count')['vel_x'].min(),left_on=['action_count'],right_index=True,how='left')['vel_x']

    df_action['v_y_mean']=df_action.merge(train_seg.groupby(by='action_count')['vel_y'].mean(),left_on=['action_count'],right_index=True,how='left')['vel_y']
    df_action['v_y_std']=df_action.merge(train_seg.groupby(by='action_count')['vel_y'].std(),left_on=['action_count'],right_index=True,how='left')['vel_y']
    df_action['v_y_max']=df_action.merge(train_seg.groupby(by='action_count')['vel_y'].max(),left_on=['action_count'],right_index=True,how='left')['vel_y']
    df_action['v_y_min']=df_action.merge(train_seg.groupby(by='action_count')['vel_y'].min(),left_on=['action_count'],right_index=True,how='left')['vel_y']

    df_action['v_t_mean']=df_action.merge(train_seg.groupby(by='action_count')['vel_total'].mean(),left_on=['action_count'],right_index=True,how='left')['vel_total']
    df_action['v_t_std']=df_action.merge(train_seg.groupby(by='action_count')['vel_total'].std(),left_on=['action_count'],right_index=True,how='left')['vel_total']
    df_action['v_t_max']=df_action.merge(train_seg.groupby(by='action_count')['vel_total'].max(),left_on=['action_count'],right_index=True,how='left')['vel_total']
    df_action['v_t_min']=df_action.merge(train_seg.groupby(by='action_count')['vel_total'].min(),left_on=['action_count'],right_index=True,how='left')['vel_total'] 

    df_action['acc_mean']=df_action.merge(train_seg.groupby(by='action_count')['acc'].mean(),left_on=['action_count'],right_index=True,how='left')['acc']
    df_action['acc_std']=df_action.merge(train_seg.groupby(by='action_count')['acc'].std(),left_on=['action_count'],right_index=True,how='left')['acc']
    df_action['acc_max']=df_action.merge(train_seg.groupby(by='action_count')['acc'].max(),left_on=['action_count'],right_index=True,how='left')['acc']
    df_action['acc_min']=df_action.merge(train_seg.groupby(by='action_count')['acc'].min(),left_on=['action_count'],right_index=True,how='left')['acc']

    df_action['jerk_mean']=df_action.merge(train_seg.groupby(by='action_count')['jerk'].mean(),left_on=['action_count'],right_index=True,how='left')['jerk']
    df_action['jerk_std']=df_action.merge(train_seg.groupby(by='action_count')['jerk'].std(),left_on=['action_count'],right_index=True,how='left')['jerk']
    df_action['jerk_max']=df_action.merge(train_seg.groupby(by='action_count')['jerk'].max(),left_on=['action_count'],right_index=True,how='left')['jerk']
    df_action['jerk_min']=df_action.merge(train_seg.groupby(by='action_count')['jerk'].min(),left_on=['action_count'],right_index=True,how='left')['jerk']

    df_action['w_mean']=df_action.merge(train_seg.groupby(by='action_count')['vel_angular'].mean(),left_on=['action_count'],right_index=True,how='left')['vel_angular']
    df_action['w_std']=df_action.merge(train_seg.groupby(by='action_count')['vel_angular'].std(),left_on=['action_count'],right_index=True,how='left')['vel_angular']
    df_action['w_max']=df_action.merge(train_seg.groupby(by='action_count')['vel_angular'].max(),left_on=['action_count'],right_index=True,how='left')['vel_angular']
    df_action['w_min']=df_action.merge(train_seg.groupby(by='action_count')['vel_angular'].min(),left_on=['action_count'],right_index=True,how='left')['vel_angular']

    df_action['curv_mean']=df_action.merge(train_seg.groupby(by='action_count')['curvat'].mean(),left_on=['action_count'],right_index=True,how='left')['curvat']
    df_action['curv_std']=df_action.merge(train_seg.groupby(by='action_count')['curvat'].std(),left_on=['action_count'],right_index=True,how='left')['curvat']
    df_action['curv_max']=df_action.merge(train_seg.groupby(by='action_count')['curvat'].max(),left_on=['action_count'],right_index=True,how='left')['curvat']
    df_action['curv_min']=df_action.merge(train_seg.groupby(by='action_count')['curvat'].min(),left_on=['action_count'],right_index=True,how='left')['curvat']

    df_action['action_type']=train_seg[train_seg.Action_border.notna()]['Action_border']
    df_action['elapsed_time']=train_seg[train_seg.Action_border.notna()]['delta_action']
    df_action['s_action']=train_seg[train_seg.Action_border.notna()]['cum_length']
    df_action['dist_end_end']=Euclidean_Dist(train_seg.loc[train_seg['begin_act_idx'].unique(),['x','y']],
    train_seg.loc[train_seg['end_act_idx'].unique(),['x','y']])
    
    vector_i_f=pd.DataFrame()
    vector_i_f=(train_seg.loc[train_seg['end_act_idx'].unique(),['x','y']].reset_index()-train_seg.loc[train_seg['begin_act_idx'].unique(),['x','y']].reset_index())
    vector_i_f['norm']=np.sqrt(vector_i_f['x']**2+vector_i_f['y']**2)

    def get_angle(data):
       if data['x']>=0  and data['y']>=0:
          return np.arccos(np.divide(data['x'],data['norm']))
       if data['x']<0  and data['y']>=0:
          return np.arccos(np.divide(data['x'],data['norm']))
       if data['x']<0  and data['y']<0:
          return 2*np.pi-np.arccos(np.divide(data['x'],data['norm']))
       if data['x']>=0 and data['y']<0:
          return 2*np.pi-np.arccos(np.divide(data['x'],data['norm']))


    def angle_to_direction(angle):
       if angle <= np.radians(45):
          return 1
       elif angle <= np.radians(90):
          return 2
       elif angle <= np.radians(135):
          return 3
       elif angle <= np.radians(180):
          return 4
       elif angle <= np.radians(225):
          return 5
       elif angle <= np.radians(270):
          return 6
       elif angle <= np.radians(315):
          return 7
       else :
          return 8
    
    
  
    
    
    #usar merge no ID
    df_action['direction']=(vector_i_f.apply(get_angle,axis=1).fillna(0)).apply(angle_to_direction).values
    df_action['straightness']=df_action['dist_end_end']/df_action['s_action']
    df_action['num_points']=train_seg[train_seg.Action_border.notna()]['index']-train_seg[train_seg.Action_border.notna()]['index'].shift().fillna(train_seg[train_seg.Action_border.notna()]['index'].values[0])
    df_action['sum_angle']= df_action.merge(train_seg.groupby(by='action_count')['theta'].sum(),left_on=['action_count'],right_index=True,how='left')['theta']
    
    
    
    df_action['largest_deviation']=df_action.merge(train_seg.groupby(by='action_count')['deviation'].max(),left_on=['action_count'],right_index=True,how='left')['deviation']
     
    df_action['sharp angles']=df_action.merge(train_seg.groupby(by='action_count')['theta'].apply(lambda x: (x<=0.0005).sum()),left_on=['action_count'],right_index=True,how='left')['theta']
    
    #from inertia aceleration time
    df_action.drop(['Unnamed: 0', 'record timestamp', 'client timestamp', 'button', 'state',
       'x', 'y','Action', 'action_begin', 'delta_action',
       'Action_border', 'begin_act_idx','act_correc_index'],inplace=True,axis=1)
    return df_action,train_seg

#obs:colocar essa função em novo notebook
#df_action,train_seg=feat_eng(train_seg)

In [188]:
train_seg=train_save
df_action,train_seg=feat_eng(train_seg[train_seg.session=='session_5815391283'])

<ipython-input-187-b1b4f9352c8d>:18: RuntimeWarning: invalid value encountered in true_divide
  d = np.divide(d_ba, (np.hypot(d_ba[:, 0], d_ba[:, 1])
<ipython-input-187-b1b4f9352c8d>:181: RuntimeWarning: invalid value encountered in true_divide
  return np.arccos(np.divide(data['x'],data['norm']))


In [189]:
df_action.isna().sum()

Unnamed: 0.1         0
session              0
user                 0
action_count         0
index                0
v_x_mean             0
v_x_std              0
v_x_max              0
v_x_min              0
v_y_mean             0
v_y_std              0
v_y_max              0
v_y_min              0
v_t_mean             0
v_t_std              0
v_t_max              0
v_t_min              0
acc_mean             0
acc_std              0
acc_max              0
acc_min              0
jerk_mean            0
jerk_std             0
jerk_max             0
jerk_min             0
w_mean               0
w_std                0
w_max                0
w_min                0
curv_mean            0
curv_std             0
curv_max             0
curv_min             0
action_type          0
elapsed_time         0
s_action             0
dist_end_end         0
direction            0
straightness         0
num_points           0
sum_angle            0
largest_deviation    0
sharp angles         0
dtype: int6

In [23]:
df_action[df_action.straightness.isna()][['dist_end_end','s_action']]

,dist_end_end,s_action
62605,0.0,0.0


In [39]:
train_seg.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'record timestamp',
       'client timestamp', 'button', 'state', 'x', 'y', 'session', 'user',
       'Action', 'action_begin', 'delta_action', 'Action_border',
       'action_count', 'index', 'begin_act_idx', 'end_act_idx', 'delta_x',
       'delta_y', 'deviation', 'dist_from_act_init', 'delta_t', 'vel_x',
       'vel_x_delta', 'vel_y', 'vel_y_delta', 'vel_total', 'vel_tot_delta',
       'theta', 'theta_delta', 'acc', 'acc_delta', 'jerk', 'vel_angular',
       'event_length', 'cum_length', 'cum_length_delta', 'curvat'],
      dtype='object')

In [54]:
train_seg.loc[:62610,:][['x','y','action_count','Action_border']]

,x,y,action_count,Action_border
62603,158,370,0.0,None
62604,158,370,0.0,None
62605,158,370,0.0,PC
62606,160,370,1.0,None
62607,161,371,1.0,None
62608,163,373,1.0,None
62609,198,383,1.0,None
62610,287,403,1.0,None


In [55]:
train_seg[train_seg.Action_border.notnull()][['action_count','cum_length']]

,action_count,cum_length
62605,0.0,0.000000e+00
62615,1.0,1.600504e+02
62617,2.0,1.600504e+02
62654,3.0,1.083187e+03
62656,4.0,1.083187e+03
...,...,...
110771,1962.0,2.929095e+06
110789,1963.0,2.929897e+06
110818,1964.0,2.932426e+06
110834,1965.0,2.933032e+06


In [80]:
train_seg=train_save
temp=pd.DataFrame()
#train_seg=train_seg[train_seg.session=='session_5265929106']
temp=pd.DataFrame((train_seg[(train_seg.Action_border=='PC')| (train_seg.Action_border=='MM') | (train_seg.Action_border=='DD')].Action_border.index))
temp['action_count']=temp.index
temp['index']=temp.loc[:,0]
temp.drop(0,axis=1,inplace= True)
temp.set_index(temp['index'],inplace=True)
temp.columns.name=None
train_seg=train_seg.merge(temp,how='left',left_index=True, right_index=True)
train_seg['action_count'].fillna(method='bfill',inplace=True)
#train_seg.drop('index',axis=1,inplace=True)
#2nd dataframe with action features
df_action=train_seg[train_seg.Action_border.notna()].copy(deep=True)
df_action['index']=train_seg[train_seg.Action_border.notna()].index
df_action['begin_act_idx']=df_action['index'].shift()
df_action['action_count']=train_seg['action_count']
df_action.loc[df_action.index[0],'begin_act_idx']=train_seg.index[0]
train_seg.loc[:,'begin_act_idx']=df_action['begin_act_idx']
###########################################################
#############################################################
train_seg.loc[:,'begin_act_idx'].fillna(method='bfill',inplace=True)
train_seg.loc[:,'end_act_idx']=df_action['index']
train_seg.loc[:,'end_act_idx'].fillna(method='bfill',inplace=True)
df_action.loc[df_action.index.to_list()[0],'begin_act_idx']=0
#delta x,y, t
train_seg['delta_x']=train_seg['x']-train_seg['x'].shift()
train_seg['delta_y']=train_seg['y']-train_seg['y'].shift()
train_seg.loc[train_seg.index[0],'delta_x']=0
train_seg.loc[train_seg.index[0],'delta_y']=0
    #print(train_seg.loc[:,'begin_act_idx'].values)
    #train_seg['delta_x_begin_act']=np.sqrt((train_seg['x'].values-train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'x'].values)**2+(train_seg['y'].values-train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'y'].values)**2)
    #train_seg['delta_x_end_act']=np.sqrt((train_seg['x'].values-train_seg.loc[train_seg.loc[:,'end_act_idx'].values,'x'].values)**2+(train_seg['y'].values-train_seg.loc[train_seg.loc[:,'end_act_idx'].values,'y'].values)**2)
    #print(np.array(list(zip(train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'x'],train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'y']))))
train_seg['deviation']=lineseg_dists(p=np.array(train_seg[['x','y']]),
                                     a=np.array(list(zip(train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'x'],train_seg.loc[train_seg.loc[:,'begin_act_idx'].values,'y']))),
                                     b=np.array(list(zip(train_seg.loc[train_seg.loc[:,'end_act_idx'].values,'x'],train_seg.loc[train_seg.loc[:,'end_act_idx'].values,'y']))))  

train_seg['dist_from_act_init']=Euclidean_Dist(train_seg.loc[train_seg['begin_act_idx'],['x','y']],
train_seg[['x','y']])
train_seg.deviation.fillna(train_seg['dist_from_act_init'],inplace=True)


#Tempo
train_seg['delta_t']=(train_seg['client timestamp']-train_seg['client timestamp'].shift())
train_seg['delta_t'].replace(0, train_seg[train_seg['delta_t']!=0].delta_t.min()/2,inplace=True)
train_seg.loc[train_seg.index[0],'delta_t']=0

#Velo
train_seg['vel_x']=train_seg['delta_x']/train_seg['delta_t']
train_seg.loc[train_seg.index[0],'vel_x']=0    
train_seg['vel_x_delta']=train_seg['vel_x']-train_seg['vel_x'].shift()
train_seg.loc[train_seg.index[0],'vel_x_delta']=0    
#velocidades 
train_seg['vel_y']=train_seg['delta_y']/train_seg['delta_t']
train_seg.loc[train_seg.index[0],'vel_y']=0    
train_seg.loc[:,'vel_y'].replace([np.inf, -np.inf], np.nan, inplace=True)
train_seg['vel_y_delta']=train_seg['vel_y']-train_seg['vel_y'].shift()
train_seg.loc[train_seg.index[0],'vel_y_delta']=0    
train_seg['vel_total']=np.sqrt(np.power(train_seg['vel_x'],2)+np.power(train_seg['vel_y'],2))
train_seg.loc[train_seg.index[0],'vel_total']=0    
train_seg['vel_tot_delta']=train_seg['vel_total']-train_seg['vel_total'].shift()
train_seg.loc[train_seg.index[0],'vel_tot_delta']=0
#theta coordenada angular
train_seg['theta']=np.arctan(train_seg['delta_y']/train_seg['delta_x'])
train_seg['theta'].replace(np.nan,0, inplace=True)
train_seg.loc[train_seg.index[0],'theta']=0
train_seg['theta_delta']=train_seg['theta']-train_seg['theta'].shift()
train_seg.loc[train_seg.index[0],'theta_delta']=0 
#extras
train_seg['acc']=train_seg['vel_tot_delta']/train_seg['delta_t']
train_seg.loc[train_seg.index[0],'acc']=0 
train_seg['acc_delta']=train_seg['acc']-train_seg['acc'].shift()
train_seg.loc[train_seg.index[0],'acc_delta']=0  
train_seg['jerk']=train_seg['acc_delta']/train_seg['delta_t']
train_seg.loc[train_seg.index[0],'jerk']=0
#velocidade angular
train_seg['vel_angular']=train_seg['theta_delta']/train_seg['delta_t']
train_seg.loc[train_seg.index[0],'vel_angular']=0
#Curvature
train_seg['event_length']=np.sqrt(np.power(train_seg['delta_x'],2)+np.power(train_seg['delta_y'],2))
train_seg['cum_length']=train_seg['event_length'].cumsum()  
train_seg['cum_length_delta']=train_seg['cum_length']-train_seg['cum_length'].shift()
train_seg.loc[train_seg.index[0],'cum_length_delta']=0   
train_seg['curvat']=train_seg['theta_delta']/train_seg['delta_t']
train_seg.loc[train_seg.index[0],'curvat']=0
#Action Dataframe
#df_action=train_seg[train_seg.Action_border.notna()].copy(deep=True)
#df_action['begin_act_idx']=[0]+df_action.index.to_list()
df_action['v_x_mean']=df_action.merge(train_seg.groupby(by='action_count')['vel_x'].mean(),left_on=['action_count'],right_index=True,how='left')['vel_x']
df_action['v_x_std']=df_action.merge(train_seg.groupby(by='action_count')['vel_x'].std(),left_on=['action_count'],right_index=True,how='left')['vel_x']
df_action['v_x_max']=df_action.merge(train_seg.groupby(by='action_count')['vel_x'].max(),left_on=['action_count'],right_index=True,how='left')['vel_x']
df_action['v_x_min']=df_action.merge(train_seg.groupby(by='action_count')['vel_x'].min(),left_on=['action_count'],right_index=True,how='left')['vel_x']
df_action['v_y_mean']=df_action.merge(train_seg.groupby(by='action_count')['vel_y'].mean(),left_on=['action_count'],right_index=True,how='left')['vel_y']
df_action['v_y_std']=df_action.merge(train_seg.groupby(by='action_count')['vel_y'].std(),left_on=['action_count'],right_index=True,how='left')['vel_y']
df_action['v_y_max']=df_action.merge(train_seg.groupby(by='action_count')['vel_y'].max(),left_on=['action_count'],right_index=True,how='left')['vel_y']
df_action['v_y_min']=df_action.merge(train_seg.groupby(by='action_count')['vel_y'].min(),left_on=['action_count'],right_index=True,how='left')['vel_y']
df_action['v_t_mean']=df_action.merge(train_seg.groupby(by='action_count')['vel_total'].mean(),left_on=['action_count'],right_index=True,how='left')['vel_total']
df_action['v_t_std']=df_action.merge(train_seg.groupby(by='action_count')['vel_total'].std(),left_on=['action_count'],right_index=True,how='left')['vel_total']
df_action['v_t_max']=df_action.merge(train_seg.groupby(by='action_count')['vel_total'].max(),left_on=['action_count'],right_index=True,how='left')['vel_total']
df_action['v_t_min']=df_action.merge(train_seg.groupby(by='action_count')['vel_total'].min(),left_on=['action_count'],right_index=True,how='left')['vel_total'] 
df_action['acc_mean']=df_action.merge(train_seg.groupby(by='action_count')['acc'].mean(),left_on=['action_count'],right_index=True,how='left')['acc']
df_action['acc_std']=df_action.merge(train_seg.groupby(by='action_count')['acc'].std(),left_on=['action_count'],right_index=True,how='left')['acc']
df_action['acc_max']=df_action.merge(train_seg.groupby(by='action_count')['acc'].max(),left_on=['action_count'],right_index=True,how='left')['acc']
df_action['acc_min']=df_action.merge(train_seg.groupby(by='action_count')['acc'].min(),left_on=['action_count'],right_index=True,how='left')['acc']
df_action['jerk_mean']=df_action.merge(train_seg.groupby(by='action_count')['jerk'].mean(),left_on=['action_count'],right_index=True,how='left')['jerk']
df_action['jerk_std']=df_action.merge(train_seg.groupby(by='action_count')['jerk'].std(),left_on=['action_count'],right_index=True,how='left')['jerk']
df_action['jerk_max']=df_action.merge(train_seg.groupby(by='action_count')['jerk'].max(),left_on=['action_count'],right_index=True,how='left')['jerk']
df_action['jerk_min']=df_action.merge(train_seg.groupby(by='action_count')['jerk'].min(),left_on=['action_count'],right_index=True,how='left')['jerk']
df_action['w_mean']=df_action.merge(train_seg.groupby(by='action_count')['vel_angular'].mean(),left_on=['action_count'],right_index=True,how='left')['vel_angular']
df_action['w_std']=df_action.merge(train_seg.groupby(by='action_count')['vel_angular'].std(),left_on=['action_count'],right_index=True,how='left')['vel_angular']
df_action['w_max']=df_action.merge(train_seg.groupby(by='action_count')['vel_angular'].max(),left_on=['action_count'],right_index=True,how='left')['vel_angular']
df_action['w_min']=df_action.merge(train_seg.groupby(by='action_count')['vel_angular'].min(),left_on=['action_count'],right_index=True,how='left')['vel_angular']
df_action['curv_mean']=df_action.merge(train_seg.groupby(by='action_count')['curvat'].mean(),left_on=['action_count'],right_index=True,how='left')['curvat']
df_action['curv_std']=df_action.merge(train_seg.groupby(by='action_count')['curvat'].std(),left_on=['action_count'],right_index=True,how='left')['curvat']
df_action['curv_max']=df_action.merge(train_seg.groupby(by='action_count')['curvat'].max(),left_on=['action_count'],right_index=True,how='left')['curvat']
df_action['curv_min']=df_action.merge(train_seg.groupby(by='action_count')['curvat'].min(),left_on=['action_count'],right_index=True,how='left')['curvat']
df_action['action_type']=train_seg[train_seg.Action_border.notna()]['Action_border']
df_action['elapsed_time']=train_seg[train_seg.Action_border.notna()]['delta_action']
df_action['s_action']=train_seg[train_seg.Action_border.notna()]['cum_length']
df_action['dist_end_end']=Euclidean_Dist(train_seg.loc[train_seg['begin_act_idx'].unique(),['x','y']],
train_seg.loc[train_seg['end_act_idx'].unique(),['x','y']])

vector_i_f=pd.DataFrame()
vector_i_f=(train_seg.loc[train_seg['end_act_idx'].unique(),['x','y']].reset_index()-train_seg.loc[train_seg['begin_act_idx'].unique(),['x','y']].reset_index())
vector_i_f['norm']=np.sqrt(vector_i_f['x']**2+vector_i_f['y']**2)

def get_angle(data):
   if data['x']>=0  and data['y']>=0:
      return np.arccos(np.divide(data['x'],data['norm']))
   if data['x']<0  and data['y']>=0:
      return np.arccos(np.divide(data['x'],data['norm']))
   if data['x']<0  and data['y']<0:
      return 2*np.pi-np.arccos(np.divide(data['x'],data['norm']))
   if data['x']>=0 and data['y']<0:
      return 2*np.pi-np.arccos(np.divide(data['x'],data['norm']))
def angle_to_direction(angle):
   if angle <= np.radians(45):
      return 1
   elif angle <= np.radians(90):
      return 2
   elif angle <= np.radians(135):
      return 3
   elif angle <= np.radians(180):
      return 4
   elif angle <= np.radians(225):
      return 5
   elif angle <= np.radians(270):
      return 6
   elif angle <= np.radians(315):
      return 7
   else :
      return 8





#usar merge no ID
df_action['direction']=(vector_i_f.apply(get_angle,axis=1).fillna(0)).apply(angle_to_direction)
df_action['straightness']=df_action['dist_end_end']/df_action['s_action']
df_action['num_points']=train_seg[train_seg.Action_border.notna()]['index']-train_seg[train_seg.Action_border.notna()]['index'].shift()
df_action['sum_angle']= df_action.merge(train_seg.groupby(by='action_count')['theta'].sum(),left_on=['action_count'],right_index=True,how='left')['theta']
    
    
    
df_action['largest_deviation']=df_action.merge(train_seg.groupby(by='action_count')['deviation'].max(),left_on=['action_count'],right_index=True,how='left')['deviation']
     
df_action['sharp angles']=df_action.merge(train_seg.groupby(by='action_count')['theta'].apply(lambda x: (x<=0.0005).sum()),left_on=['action_count'],right_index=True,how='left') ['theta']
    
#from inertia aceleration time
df_action.drop(['Unnamed: 0', 'record timestamp', 'client timestamp', 'button', 'state',
    'x', 'y','Action', 'action_begin', 'delta_action',
    'Action_border', 'begin_act_idx'],inplace=True,axis=1)


<ipython-input-76-7a93bbe42de9>:18: RuntimeWarning: invalid value encountered in true_divide
  d = np.divide(d_ba, (np.hypot(d_ba[:, 0], d_ba[:, 1])
<ipython-input-80-9171fe4c767d>:130: RuntimeWarning: invalid value encountered in true_divide
  return np.arccos(np.divide(data['x'],data['norm']))


In [81]:
train_seg[train_seg.Action_border.notna()]['index']-train_seg[train_seg.Action_border.notna()]['index'].shift().fillna(train_seg[train_seg.Action_border.notna()]['index'].values[0])

30292     NaN
30297     5.0
30299     2.0
30323    24.0
30333    10.0
         ... 
62551    21.0
62559     8.0
62570    11.0
62585    15.0
62602    17.0
Name: index, Length: 1482, dtype: float64

In [82]:
train_seg[train_seg.Action_border.notna()]['index']

30292    30292.0
30297    30297.0
30299    30299.0
30323    30323.0
30333    30333.0
          ...   
62551    62551.0
62559    62559.0
62570    62570.0
62585    62585.0
62602    62602.0
Name: index, Length: 1482, dtype: float64

In [90]:
train_seg[train_seg.Action_border.notna()]['index'].shift().fillna(train_seg[train_seg.Action_border.notna()]['index'].values[0])

30292    30292.0
30297    30292.0
30299    30297.0
30323    30299.0
30333    30323.0
          ...   
62551    62530.0
62559    62551.0
62570    62559.0
62585    62570.0
62602    62585.0
Name: index, Length: 1482, dtype: float64

In [89]:
train_seg[train_seg.Action_border.notna()]['index'].values[0]

30292.0